# Paris Subway Data

This will create a simple model of Paris Subway.

Based on the RATP open dataset  (the transport company in Paris) : https://dataratp.opendatasoft.com/

The dataset can be retrieve using this link : https://dataratp.opendatasoft.com/explore/dataset/offre-transport-de-la-ratp-format-gtfs/information/

All possibilities will not be take into account. This is only for testing purpose. Feel free to improve it.

In [1]:
from dataclasses import dataclass
import pandas as pd
from hashlib import sha1
from jinja2 import Template
import dateutil.parser as dateparser
from datetime import datetime, timedelta

# Force some reset ...
pd.reset_option("display.max_rows")

### Constants

In [2]:
OUTPUT_FILE = 'paris-subway.cypher'
SUBWAY_LINES = list(map(str, range(1,15))) + ['3B', '7B']
# SUBWAY_LINES = ['10', '14', '3']

# Show result
#SUBWAY_LINES

### Functions

* `hash_str` : produce a hexa sha1 format of a string
* `date_parser` : parse a date 
  * `24:15:35` => `datetime(2020, 1, 1, 0, 15, 35)`
  * `00:15:35` => `datetime(2020, 1, 1, 0, 15, 35)`
* `diff_second` : calculate the second between 2 dates
  * `to_time=date_parser('24:15:35'), from_time=date_parser('24:14:35')` => `60`
  * `to_time=date_parser('00:02:35'), from_time=date_parser('23:59:35')` => `180`

In [3]:
def hash_str(value: str):
    return sha1(str(value).encode('utf-8')).hexdigest()

def date_parser(value: str):
    p = value.split(':')
    d = dateparser.parse(f'{(int(p[0]) % 24):02}:{(int(p[1]) % 60):02}:{(int(p[2]) % 60):02}')
    return d.replace(year=2020, month=1, day=1)

def diff_second(to_time, from_time):
    if to_time < from_time:
        to_time = to_time + timedelta(days=1)
    return int((to_time - from_time).total_seconds())

## Loading CSV Data

Each CSV will be load. Data which will no be used will be drop.

### Load CSV : `routes.txt` : `routes_df`

* Drop columns : `['agency_id', 'route_desc', 'route_url', 'route_color', 'route_text_color', 'route_type']`
* Filter on : `SUBWAY_LINES`
* Reset index

In [4]:
routes_df = pd.read_csv(
    'routes.txt',
    dtype={ 'route_id': int, 'route_short_name': str, 'route_long_name': str }
)
routes_df = routes_df.drop(
    columns=['agency_id', 'route_desc', 'route_url', 'route_color', 'route_text_color', 'route_type']
)

if SUBWAY_LINES:
    routes_df = routes_df[( routes_df['route_short_name'].isin(SUBWAY_LINES) )]

routes_df = routes_df.reset_index(drop=True)
routes_df.head(5)

,route_id,route_short_name,route_long_name
0,2272462,5,(BOBIGNY - PABLO PICASSO <-> PLACE D'ITALIE) -...
1,2272463,5,(BOBIGNY - PABLO PICASSO <-> PLACE D'ITALIE) -...
2,2272466,7,(LA COURNEUVE - 8 MAI 1945 <-> VILLEJUIF-L. AR...
3,2272467,7,(LA COURNEUVE - 8 MAI 1945 <-> VILLEJUIF-L. AR...
4,2272468,7,(LA COURNEUVE - 8 MAI 1945 <-> VILLEJUIF-L. AR...


### Load CSV `trips.txt` : `trips_df`

* Drop columns : `['service_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'shape_id']`
* Merge with `routes_df` on key `route_id`
* Reset index

In [5]:
trips_df = pd.read_csv(
    'trips.txt',
    dtype={'route_id':int, 'trip_id':int, 'trip_headsign':str, 'trip_short_name':str}
)
trips_df = trips_df.drop(columns=['service_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'shape_id'])
trips_df = pd.merge(trips_df, routes_df, on='route_id').reset_index(drop=True)
trips_df

,route_id,trip_id,route_short_name,route_long_name
0,2272462,26231428312203953,5,(BOBIGNY - PABLO PICASSO <-> PLACE D'ITALIE) -...
1,2272462,26331428312203953,5,(BOBIGNY - PABLO PICASSO <-> PLACE D'ITALIE) -...
2,2272462,26431428312203953,5,(BOBIGNY - PABLO PICASSO <-> PLACE D'ITALIE) -...
3,2272462,26531428312203953,5,(BOBIGNY - PABLO PICASSO <-> PLACE D'ITALIE) -...
4,2272462,26631428312203953,5,(BOBIGNY - PABLO PICASSO <-> PLACE D'ITALIE) -...
...,...,...,...,...
108670,2416131,9534576491078872,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Retour
108671,2416131,9634576491078872,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Retour
108672,2416131,9734576491078872,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Retour
108673,2416131,9834576491078872,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Retour


### Load `stop_times.txt` : `stop_times_df`

<div class="alert alert-block alert-warning">
This file may take a while to be loaded. `stop_times.txt` is pretty large (currently: `616.81MB`)
</div>

In this file, all trips are describe with the arrival and departure times in each stops.

* Drop columns : `['stop_headsign', 'shape_dist_traveled']`
* Merge with `trips_df` on key `trip_id`
* Transform columns
  * `arrival_time` into `datetime`
  * `departure_time` into `datetime`
* Reset index

In [6]:
stop_times_df = pd.read_csv(
    'stop_times.txt',
    dtype={'trip_id':int, 'stop_id':int, 'arrival_time':str, 'departure_time':str}
)
stop_times_df = stop_times_df.drop(columns=['stop_headsign', 'shape_dist_traveled'])
stop_times_df = pd.merge(stop_times_df, trips_df, on='trip_id')
stop_times_df['arrival_time'] = stop_times_df['arrival_time'].apply(date_parser)
stop_times_df['departure_time'] = stop_times_df['departure_time'].apply(date_parser)
stop_times_df = stop_times_df.reset_index(drop=True)
stop_times_df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_id,route_short_name,route_long_name
0,10031412331078871,2020-01-01 14:01:00,2020-01-01 14:01:00,2035,1,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller
1,10031412331078871,2020-01-01 14:03:00,2020-01-01 14:03:00,2067,2,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller
2,10031412331078871,2020-01-01 14:04:00,2020-01-01 14:04:00,1725,3,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller
3,10031412331078871,2020-01-01 14:06:00,2020-01-01 14:06:00,1751,4,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller
4,10031412331078871,2020-01-01 14:07:00,2020-01-01 14:07:00,1832,5,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller
...,...,...,...,...,...,...,...,...
2617116,9934576502416128,2020-01-01 12:48:00,2020-01-01 12:48:00,2110,23,2299375,4,(PORTE DE CLIGNANCOURT <-> MAIRIE DE MONTROUGE...
2617117,9934576502416128,2020-01-01 12:49:00,2020-01-01 12:49:00,2152,24,2299375,4,(PORTE DE CLIGNANCOURT <-> MAIRIE DE MONTROUGE...
2617118,9934576502416128,2020-01-01 12:50:00,2020-01-01 12:50:00,2535,25,2299375,4,(PORTE DE CLIGNANCOURT <-> MAIRIE DE MONTROUGE...
2617119,9934576502416128,2020-01-01 12:51:00,2020-01-01 12:51:00,2478,26,2299375,4,(PORTE DE CLIGNANCOURT <-> MAIRIE DE MONTROUGE...


### Load `stops.txt` : `stops_df`

* Load CSV `stops.txt`
* Drop columns : `['stop_code', 'stop_desc', 'location_type', 'parent_station']`
* Merge with `stop_times_df['stop_id']` on `stop_id`
* Drop duplicates
* Reset index

In [7]:
stops_df = pd.read_csv(
    'stops.txt',
    dtype={'trip_id':int, 'stop_id':int, 'trip_headsign':str, 'trip_short_name':str}
)
stops_df = stops_df.drop(columns=['stop_code', 'stop_desc', 'location_type', 'parent_station'])
stops_df["station_id"] = stops_df["stop_name"].apply(hash_str)
stops_df = pd.merge(stops_df, stop_times_df['stop_id'], on=['stop_id']).drop_duplicates()
stops_df = stops_df.reset_index(drop=True)
stops_df = stops_df.rename(columns={
    'stop_name': 'name',
    'stop_lat': 'lat',
    'stop_lon': 'lon'
})
stops_df.head(5)

,stop_id,name,lat,lon,station_id
0,1166824,Olympiades,48.826948,2.367038,c013a983778accb3494ea2b6e2d62d9b364f53f6
1,1166825,Olympiades,48.826948,2.367038,c013a983778accb3494ea2b6e2d62d9b364f53f6
2,1166826,Bibliothèque-François Mitterrand,48.829831,2.376120,e799ff89f02653528c8660989938659f86e200ec
3,1166827,Bibliothèque-François Mitterrand,48.829831,2.376120,e799ff89f02653528c8660989938659f86e200ec
4,1166828,Cour Saint-Emilion,48.833521,2.386265,757558be993d3632918145952a6b1e785d75c347


### Load `transfers.txt` : `transfers_df`

* Load CSV `transfers_df`
* Reject all values where :
  * `from_stop_id` don't belong to `stops_df['stop_id']`
  * `to_stop_id` don't belong to `stops_df['stop_id']`
* Reset index

In [8]:
transfers_df = pd.read_csv(
    'transfers.txt',
    dtype={'from_stop_id':int, 'to_stop_id':int, 'transfer_type':int, 'min_transfer_time':int}
)
transfers_df = transfers_df[( transfers_df['from_stop_id'].isin(stops_df['stop_id']) )]
transfers_df = transfers_df[( transfers_df['to_stop_id'].isin(stops_df['stop_id']) )]
transfers_df = transfers_df.reset_index(drop=True)
transfers_df.head(5)

,from_stop_id,to_stop_id,transfer_type,min_transfer_time
0,1166832,1955,2,420
1,1166832,2210,2,240
2,1166833,1955,2,420
3,1166833,2210,2,240
4,1166834,1965,2,420


## Transformations

Apply some transformation on data to generate a simple cypher for Neo4J

### Lines : `lines_df`

* Based on `routes_df`
* Take only column `route_short_name`
* Drop duplicates
* Rename columns `route_short_name` into `name`
* Integrate hash for line

In [9]:
lines_df = routes_df['route_short_name'].to_frame()
lines_df = lines_df.drop_duplicates()
lines_df = lines_df.rename(columns={'route_short_name': 'name'})
lines_df['line_id'] = lines_df['name'].apply(hash_str) 
lines_df.head(5)

,name,line_id
0,5,ac3478d69a3c81fa62e60f5c3696165a4e5e6ac4
2,7,902ba3cda1883801594b6e1b452790cc53948fda
6,7B,c274b3c0dad677526355847743c36dfa47899f46
8,9,0ade7c2cf97f75d009975f4d720d1fa6c19f4897
10,10,b1d5781111d84f7b3fe45a0852e59758cd7a87e5


### Stations : `stations_df`

* Based on `stop_times_df` & `stops_df`
* Drop columns `stop_id` of `stops_df`
* Rename columns
* Drop duplicates
* Group by `station_id` and aggregate `line`
* Drop columns `line`
* Drop duplicate based on `station_id`
* Reset index

In [10]:
stations_df = stop_times_df.drop(columns=['trip_id', 'arrival_time', 'departure_time', 'stop_sequence'])
stations_df = pd.merge(stops_df, stations_df, on=['stop_id'])
stations_df = stations_df.drop(columns=['route_id', 'route_long_name', 'stop_id'])
stations_df = stations_df.rename(columns={'route_short_name': 'line'})
stations_df = stations_df.drop_duplicates()
_tmp = stations_df.groupby(['station_id']).agg(lines=('line', list)).reset_index()
stations_df = stations_df.drop(columns=['line']).drop_duplicates()
stations_df = pd.merge(stations_df, _tmp, on=['station_id'])
del _tmp
stations_df = stations_df.drop_duplicates(subset=['station_id']).reset_index()
stations_df.head(10)

,index,name,lat,lon,station_id,lines
0,0,Olympiades,48.826948,2.367038,c013a983778accb3494ea2b6e2d62d9b364f53f6,[14]
1,1,Bibliothèque-François Mitterrand,48.829831,2.376120,e799ff89f02653528c8660989938659f86e200ec,[14]
2,2,Cour Saint-Emilion,48.833521,2.386265,757558be993d3632918145952a6b1e785d75c347,[14]
3,3,Bercy,48.840543,2.379409,51cac4e9016ad7dc7f036283af508252b245b360,"[14, 6]"
4,5,Gare de Lyon,48.844652,2.373108,49541b7895349f795ad38c3b7fc249aec6b490b7,"[14, 1]"
5,7,Châtelet,48.858956,2.347361,ae9c8e1611f553fd7dc21d399dc4161a788ca201,"[14, 4, 11, 7, 1]"
6,12,Madeleine,48.869579,2.324162,3f6701fa5352e7df067b97a9ae7c9131d57f6472,"[14, 12, 8]"
7,15,Saint-Lazare,48.875538,2.325196,9312306d63bb3b64ed9d6092ec90f7c194d27a93,"[14, 3, 12, 13]"
8,19,Saint-Denis-Université,48.946111,2.362045,f85e6dfb1bc7fad3573a86527a6e1f6f49662be4,[13]
9,20,Villiers,48.881324,2.316597,33a3c8c64eb4fe122a6a0e3ee083ac2018576639,"[2, 3]"


### Filter complete routes : `complete_stop_times_df`

Skip all routes that may not be complete

On partial routes :
* a stop may be skipped
* the terminus can be in the middle of the lines

In [11]:
_tmp_stops = stops_df.drop(columns=['name', 'lat', 'lon'])
route_having_max_seq_series = stop_times_df.groupby('route_id')['stop_sequence'].idxmax()
complete_route_id_series = stop_times_df.loc[route_having_max_seq_series]['route_id']
complete_stop_times_df = pd.merge(stop_times_df, complete_route_id_series, on=['route_id'])
complete_stop_times_df = pd.merge(complete_stop_times_df, _tmp_stops, on=['stop_id'])
del _tmp_stops
complete_stop_times_df.head(5)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_id,route_short_name,route_long_name,station_id
0,10031412331078871,2020-01-01 14:01:00,2020-01-01 14:01:00,2035,1,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller,9e23e8f481e69e44a5d39b27f4f664061aa934f7
1,10031412351078871,2020-01-01 14:15:00,2020-01-01 14:15:00,2035,1,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller,9e23e8f481e69e44a5d39b27f4f664061aa934f7
2,10031412391078871,2020-01-01 09:13:00,2020-01-01 09:13:00,2035,1,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller,9e23e8f481e69e44a5d39b27f4f664061aa934f7
3,10031412401078871,2020-01-01 09:19:00,2020-01-01 09:19:00,2035,1,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller,9e23e8f481e69e44a5d39b27f4f664061aa934f7
4,10031412441078871,2020-01-01 13:10:00,2020-01-01 13:10:00,2035,1,2416130,1,(CHATEAU DE VINCENNES <-> LA DEFENSE) - Aller,9e23e8f481e69e44a5d39b27f4f664061aa934f7


### Travel between 2 stops : `stop_travels_df`

<div class="alert alert-block alert-warning">
This can take time... Based on `complete_stop_times_df`, many operations will be done.
</div>

* Based on `complete_stop_times_df`
* Create 2 DataFrames :
  * `_from_stops_df` : where we came from (rename columns)
  * `_to_stops_df` : where we are going to (rename columns)
* Merge this 2 DataFrames to create a new one : `stop_travels_df`
* Calculate the :
  * `arrival_duration` : Diff of seconds between stops based on the `arrival_time`
  * `departure_duration` : Diff of seconds between stops based on the `departure_time`

In [12]:
_base_stops_df = complete_stop_times_df.drop(columns=['route_long_name'])

# From stops
_from_stops_df = _base_stops_df.rename(
    columns={
        'stop_id': 'from_stop_id',
        'stop_name': 'from_stop_name',
        'station_id': 'from_station_id',
        'stop_sequence': 'from_stop_sequence',
        'arrival_time': 'from_arrival_time',
        'departure_time': 'from_departure_time',
    }
)
_from_stops_df['to_stop_sequence'] = _from_stops_df['from_stop_sequence'] + 1

# Next stops
_to_stops_df = _base_stops_df.drop(columns=['route_id', 'route_short_name'])
_to_stops_df = _to_stops_df.rename(
    columns={
        'stop_id': 'to_stop_id',
        'stop_name': 'to_stop_name',
        'station_id': 'to_station_id',
        'stop_sequence': 'to_stop_sequence',
        'arrival_time': 'to_arrival_time',
        'departure_time': 'to_departure_time',
    }
)

# Merge
stop_travels_df = pd.merge(_from_stops_df, _to_stops_df, on=[ 'trip_id', 'to_stop_sequence'])

# Clean
del _base_stops_df
del _from_stops_df
del _to_stops_df

# Calculate duration diffs
_arrival_diff = lambda x: diff_second(x.to_arrival_time, x.from_arrival_time)
_departure_diff = lambda x: diff_second(x.to_departure_time, x.from_departure_time)
stop_travels_df['arrival_duration'] = stop_travels_df.apply(_arrival_diff, axis=1)
stop_travels_df['departure_duration'] = stop_travels_df.apply(_departure_diff, axis=1)

# Remove useles columns
stop_travels_df = stop_travels_df.drop(columns=[
    'from_stop_sequence', 'from_arrival_time', 'from_departure_time',
    'to_stop_sequence', 'to_arrival_time', 'to_departure_time'
])
stop_travels_df = stop_travels_df.reset_index(drop=True)

# Clean
del _arrival_diff
del _departure_diff

stop_travels_df.head(5)

,trip_id,from_stop_id,route_id,route_short_name,from_station_id,to_stop_id,to_station_id,arrival_duration,departure_duration
0,10031412331078871,2035,2416130,1,9e23e8f481e69e44a5d39b27f4f664061aa934f7,2067,5b02b0e7ad4f13515b5b21e41413a312bb3a1656,120,120
1,10031412351078871,2035,2416130,1,9e23e8f481e69e44a5d39b27f4f664061aa934f7,2067,5b02b0e7ad4f13515b5b21e41413a312bb3a1656,120,120
2,10031412391078871,2035,2416130,1,9e23e8f481e69e44a5d39b27f4f664061aa934f7,2067,5b02b0e7ad4f13515b5b21e41413a312bb3a1656,120,120
3,10031412401078871,2035,2416130,1,9e23e8f481e69e44a5d39b27f4f664061aa934f7,2067,5b02b0e7ad4f13515b5b21e41413a312bb3a1656,60,60
4,10031412441078871,2035,2416130,1,9e23e8f481e69e44a5d39b27f4f664061aa934f7,2067,5b02b0e7ad4f13515b5b21e41413a312bb3a1656,60,60


### Travels : `travels_df`

* Based on `stop_travels_df`
* Grouping by `from_stop_id` and `to_stop_id`
* Calculate the mean of duration between the stop `from_stop_id` and `to_stop_id`
* Remove columns `['trip_id', 'arrival_duration', 'departure_duration']`
* Drop duplicates


In [13]:
# Calculate mean time between 2 stops
_stop_means = stop_travels_df.groupby(['from_stop_id', 'to_stop_id'])[['arrival_duration', 'departure_duration']].mean()
_stop_means = _stop_means.reset_index()
_stop_means['mean_transfer_time'] = _stop_means[['arrival_duration', 'departure_duration']].mean(axis=1)
_stop_means['mean_transfer_time'] = _stop_means['mean_transfer_time'].astype(int)
_stop_means = _stop_means.drop(columns=['arrival_duration', 'departure_duration'])
# Merge & Clean
travels_df = pd.merge(stop_travels_df, _stop_means, on=[ 'from_stop_id', 'to_stop_id'])
travels_df = travels_df.drop(columns=['trip_id', 'arrival_duration', 'departure_duration'])
travels_df = travels_df.drop_duplicates().reset_index(drop=True)
travels_df

,from_stop_id,route_id,route_short_name,from_station_id,to_stop_id,to_station_id,mean_transfer_time
0,2035,2416130,1,9e23e8f481e69e44a5d39b27f4f664061aa934f7,2067,5b02b0e7ad4f13515b5b21e41413a312bb3a1656,79
1,2035,2416130,1,9e23e8f481e69e44a5d39b27f4f664061aa934f7,1832,135c7e55f428deec34db86894ff054ab0a22900a,300
2,2067,2416130,1,5b02b0e7ad4f13515b5b21e41413a312bb3a1656,1725,4b8233fad181fd35e2999083df9010dc57b30b3e,73
3,1725,2416130,1,4b8233fad181fd35e2999083df9010dc57b30b3e,1751,3784adacc9a6d443571a3b9701c84ee987710823,108
4,1751,2416130,1,3784adacc9a6d443571a3b9701c84ee987710823,1832,135c7e55f428deec34db86894ff054ab0a22900a,98
...,...,...,...,...,...,...,...
911,2018,2278414,10,c3e0633bc8abcbd5f4b5fa674ea4a4f015ceaad6,1908,472fe5b84e856bbef75af57174a2c080d15c3de0,70
912,2018,2278415,10,c3e0633bc8abcbd5f4b5fa674ea4a4f015ceaad6,1908,472fe5b84e856bbef75af57174a2c080d15c3de0,70
913,1908,2278414,10,472fe5b84e856bbef75af57174a2c080d15c3de0,1952,92b9bb5007dec1d038f09121dfaffc7e3f05da2b,170
914,1908,2278415,10,472fe5b84e856bbef75af57174a2c080d15c3de0,1952,92b9bb5007dec1d038f09121dfaffc7e3f05da2b,170


## Format with Jinja into Neo4J Cypher

In [14]:
stations_df.head(2).to_dict('records')

[{'index': 0,
  'name': 'Olympiades',
  'lat': 48.82694828196075,
  'lon': 2.367038433387592,
  'station_id': 'c013a983778accb3494ea2b6e2d62d9b364f53f6',
  'lines': ['14']},
 {'index': 1,
  'name': 'Bibliothèque-François Mitterrand',
  'lat': 48.82983069527111,
  'lon': 2.3761200093448367,
  'station_id': 'e799ff89f02653528c8660989938659f86e200ec',
  'lines': ['14']}]

In [15]:
def _tpl(df, name: Template):
    return df.apply(lambda r: name.render(r.to_dict()), axis='columns').str.cat(sep=',\n')

_station_nodes_tpl = Template('''\
  (STATION_{{ station_id }}:Station { name: "{{ name }}", latitude: {{ lat }}, longitude: {{ lon }}, lines: {{ lines }} })
''')

_stop_nodes_tpl = Template('''\
  (STOP_{{ stop_id }}:Stop { name: "{{ name }}", stop_id: "{{ stop_id }}" })
''')

_stop_station_relationships_tpl = Template('''\
  (STOP_{{ stop_id }})-[:LOCATED]->(STATION_{{ station_id }})
''')

_travel_relationships_tpl = Template('''\
  (STOP_{{ from_stop_id }})-[:TRAVEL_TO { line: "{{ route_short_name }}", transfer_time: {{ mean_transfer_time }} }]->(STOP_{{ to_stop_id }})
''')

_transfer_relationships_tpl = Template('''\
  (STOP_{{ from_stop_id }})-[:TRANSFER { transfer_time: {{ min_transfer_time }} }]->(STOP_{{ to_stop_id }})
''')


CYPHER_TEMPLATE = '''\
// Based on https://dataratp.opendatasoft.com/explore/dataset/offre-transport-de-la-ratp-format-gtfs/information/

// ============
// NODES
// ============

// Create Stations
CREATE
{{ stations }}

// Create Stops
CREATE
{{ stops }}

// ============
// RELATIONS
// ============

// Create Stop stations
CREATE 
{{ stop_stations }}

// Create Travels
CREATE 
{{ travels }}

// Create Transfers
CREATE 
{{ transfers }}
'''

Template(CYPHER_TEMPLATE). \
    stream(
        stations=_tpl(stations_df, _station_nodes_tpl),
        stops=_tpl(stops_df, _stop_nodes_tpl),
        travels=_tpl(travels_df, _travel_relationships_tpl),
        transfers=_tpl(transfers_df, _transfer_relationships_tpl),
        stop_stations=_tpl(stops_df, _stop_station_relationships_tpl),
    ). \
    dump(OUTPUT_FILE)

# Show result
with open(OUTPUT_FILE) as fd:
    content = fd.read()
    print(content)

// Based on https://dataratp.opendatasoft.com/explore/dataset/offre-transport-de-la-ratp-format-gtfs/information/

// ============
// NODES
// ============

// Create Stations
CREATE
  (STATION_c013a983778accb3494ea2b6e2d62d9b364f53f6:Station { name: "Olympiades", latitude: 48.82694828196075, longitude: 2.367038433387592, lines: ['14'] }),
  (STATION_e799ff89f02653528c8660989938659f86e200ec:Station { name: "Bibliothèque-François Mitterrand", latitude: 48.82983069527111, longitude: 2.3761200093448367, lines: ['14'] }),
  (STATION_757558be993d3632918145952a6b1e785d75c347:Station { name: "Cour Saint-Emilion", latitude: 48.83352086080552, longitude: 2.3862652367404573, lines: ['14'] }),
  (STATION_51cac4e9016ad7dc7f036283af508252b245b360:Station { name: "Bercy", latitude: 48.84054278296546, longitude: 2.3794094631230687, lines: ['14', '6'] }),
  (STATION_49541b7895349f795ad38c3b7fc249aec6b490b7:Station { name: "Gare de Lyon", latitude: 48.84465215098295, longitude: 2.3731081475452798, line